DATAINGESTION-TRAINING STEPS:-

config/config.yaml , params.yaml , dvc.yaml ------ tests/unit/integration tests

src/deepClassifier/-config-entity-constants-utils

dataingestion
preparingmode               all three goes to Training --- Best Model Selector
prepare Callbacks 



Constants-init- has 
CONFIG_FILE_PATH,PARAMS_FILE_PATH = paths of both these

# STAGE1:DATA-INGESTION:
Config.yaml: data_ingestion: this is key for neseted key-value pair of 
root_dir:,source_url:,local_data_file:,unzip_dir:

## named tuple creation:by class method
1.DataIngestionConfig: class : has class-variables:root_dir,source_URL,local_data_file,unzip_dir
2.ConfigurationManager class :
   1. init taken config_file_path and param_file_path.
   2. create root directory /artifacts from utility we imported create_directories class
   3.get_data_ingestion_config() : class : created root directory for this point and merges config.yaml values with named tuple keys
3. COMPONENT: 
    DataIngestion:
        1. init has config:DataIngestionConfig
        2. download_file(): function : if datafile doesnot exists in the system then only do the download, with urllib.request lib retrieved the data
        3. unzip_and_clean(): function : reading the zip file and cleaning the files such as other than jpeg format and 0kb size file will be removed for that filename.namelist() : function : helps to find names of images and also used hidden method _get_updated_list_of_files() : function : to remove those unwanted files, _preprocess(): hidden method function : size is checked and removed

        4. _get_updated_list_of_files() : function : here given conditions: endswith jpeg and names of file is cat or dog then only select those files.

        5. _preprocess() : function : to remove 0kb size file.
4. pipeline created : by taking ConfigurationManager, take all functions in component and ConfigManager.
    get_data_ingestion_config(),download_file(),unzip_and_clean()

# STAGE-2: PrepareBaseModelConfig:

config.yaml: has prepare_base_model key with key-value pairs of
root_dir: artifact dir,base_model_path:model.h5,updated_base_model_path:transfer learning model.

# named tuple by dataclass decorator method
1. PrepareBaseModelConfig: class : 
    has type casting of root_dir:, base_model_path:,updated_base_model_path:, params_image_size:,params_classes:,params_learning_rate:,params_include_top:,params_weights:
2. ConfigurationManager class :
    1. get_prepare_base_model_config(): function : creating directory for this stage and get config details from data class method,provide values by joining with the  config.yaml.
3. Component:
    PrepareBaseModel(): class : has 
    1. init has config: named tuple PrepareBaseModelConfig
    # visual-geometric-group
    2. get_base_model() : function : has model object of tf.keras.applications.vgg16.VGG16(input_shape,weights,include_top)
    # Include top =True cnn+ann, false = only cnn
    # params.yaml 
    base model path is defined and used in save_model() function 
        @staticmethod
    3. _prepare_full_model(): if freeze_all is selected then with forloop make all layers untrainable, else use freeze till option
    then apply flatten operation by 
    flatten_in= tf.keras.layers.Flatten()(model.output)
    prediction layer added = tf.keras.layers.Dense(units=classes,activation='softmax)(flatten_in)
    # (flatten_in) this is input for prediction layer 
    full_model = tf.keras.models.Model(inputs=model.input,output = prediction)
    full_modile.compile()
    4. update_base_model(): function : full_model is defined that has attributes basemodel, classes,freeze_all :freezing all layers, freeze_till: till what layer :given None ,learning_rate
    self.save_model(path,model=self.full_model)
       @staticmethod
    5. save_model(): function : it takes model, path as input variables and save it
4. pipeline :
     has configurationManager function get_prepare_base_model_config() and component functions  get_base_model(),update_base_model(),save_model()

# PARAMS.yaml
1.AUGUMENTATION: TRUE (if data is not sufficient go with this this multiply same data by changing minute things in that.)
2. IMAGE_SIZE : [224,224,3] vgg16 model takes input as this size
3. INCLUDE_TOP : FALSE we are using only CNN layer here.
4. EPOCHS:1 only one cycle we are running as we are doing testing 
5. CLASSES :2  cat/dog are 2 classes
6. WEIGHTS: imagenet ( imagenet is a dataset , in competition,it has 1000 no of classes(animals) it has wide variety of trained weights)
7. LEARNING_RATE : 0.01


# stage:3 -Prepare callback

config.yaml : prepare_callbacks: root_dir,
tensorboard_root_log_dir,checkpoint_model_file_path key-value pairs.

1. dataclass method : namedtuple:
    PrepareCallbackConfig: class : has rootdir,tensorboard_root_log_dir,checkpoint_model_file_path
2. ConfigManager(): class: linking config.yaml and dataclass named tuple with the help of PrepareCallbacksConfig() function.
3. Component:
    PrepareCallback(): class : has 
     1. init takes PrepareCallbacksConfig function from configurationManager.
     2. get_to_ckpt_callbacks() function that returns hidden functions i.e. _create_tb_callbacks() , _create_ckpt_callbacks().
     3. _create_tb_callbacks() : function : dir details with time stamp, it has log files we can use for visualisation later
     4. _create_ckpt_callbacks() : it saves best weights based on best loss , saves in model.h5 .
